In [4]:
!cp "/Users/iluzaangirova/Desktop/Bert/train_predict/modeling.py" .
!cp "/Users/iluzaangirova/Desktop/Bert/train_predict/optimization.py" .
!cp "/Users/iluzaangirova/Desktop/Bert/tokenization.py" .
import tokenization 
import torch
import sys 
import time
import numpy as np
import pandas as pd
from IPython.display import clear_output
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from modeling import BertConfig, BertForSequenceClassification
from optimization import BERTAdam
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score, matthews_corrcoef

import matplotlib.pyplot as plt
import tkinter as tk


df = pd.read_csv('/Users/iluzaangirova/Desktop/Bert/dataset_5.csv', encoding='utf8')
df = df[['topics', 'title']]

maps = pd.factorize(df.topics)[1]
print(maps)
df['topics'] = pd.factorize(df.topics)[0]
print(df.head())
df['title'] = df['title'].astype('str')


device = 'cpu'
bert_config = BertConfig.from_json_file('/Users/iluzaangirova/Desktop/Bert/train_predict/bert_config.json')
model = BertForSequenceClassification(bert_config, 5)
model.to(device)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('/Users/iluzaangirova/Desktop/Bert/weights5000.pth'\
                                      , map_location='cpu'))

Index(['sport', 'society', 'auto', 'business', 'economic'], dtype='object')
   topics                                              title
0       0  Суд американского города Торранс приступит к р...
1       0  Состояние здоровья знаменитого канадского хокк...
2       0  Финский "Йокерит" потеряет преимущество своей ...
3       0  "Сакраменто" проиграл "Мемфису" в матче чемпио...
4       0  ФК "Валенсия": сделаем все, чтобы найти фаната...


<All keys matched successfully>

In [5]:
tokenizer = tokenization.FullTokenizer(vocab_file='/Users/iluzaangirova/Desktop/Bert/vocab.txt', do_lower_case=False)

In [6]:
def predict(sentence, tokenizer, max_len):
#     print(sentence)
    tokens_a = tokenizer.tokenize(sentence)
    if len(tokens_a) > max_len - 2:
        tokens_a = tokens_a[0:(max_len - 2)]

    tokens = []
    tokens.append("[CLS]")
    for token in tokens_a:
        tokens.append(token)
    tokens.append("[SEP]")
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    while len(input_ids) < max_len:
        input_ids.append(0)
        input_mask.append(0)
    input_ids = torch.LongTensor(input_ids).unsqueeze(0)
    input_mask = torch.LongTensor(input_mask).unsqueeze(0)
    a = time.time()

    logits = model(input_ids.cpu(), None, input_mask.cpu())
    logits = logits.squeeze(0)
    
    b = time.time()
    logits = F.softmax(logits, dim=-1)
    clas = logits.argmax().item()
    return clas

In [7]:
window = tk.Tk()
window.title('Classifier')
label = tk.Label(text="Введите заголовок статьи")
entry = tk.Entry()
label.pack()
entry.pack()
title1 = entry.get()
def button():
    label = tk.Label(window, text=maps[predict(entry.get(), tokenizer, 512)])
    label.pack()
btn1 = tk.Button(window, text = 'классифицировать', command = button)
btn1.pack()
window.mainloop()